In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess the dataset
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    # Convert the Date column to datetime type
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    # Use 'Close' price for forecasting
    data = df.filter(['Close'])
    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0,1))
    scaled_data = scaler.fit_transform(data)
    return scaled_data, scaler

In [ ]:
# Split the data into training and testing sets
def create_training_testing_data(scaled_data, training_data_len):
    # Create the training data set
    train_data = scaled_data[0:training_data_len, :]
    # Split the data into x_train and y_train datasets
    x_train, y_train = [], []
    for i in range(60, len(train_data)):
        x_train.append(train_data[i-60:i, 0])
        y_train.append(train_data[i, 0])
    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
    return x_train, y_train


In [ ]:
# Build the LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [ ]:
# Main function to run the stock price forecasting
def run_stock_price_forecasting(file_path):
    scaled_data, scaler = load_and_preprocess_data(file_path)
    training_data_len = int(np.ceil( len(scaled_data) * .95 ))
    
    x_train, y_train = create_training_testing_data(scaled_data, training_data_len)
    
    model = build_lstm_model((x_train.shape[1], 1))
    # Train the model
    model.fit(x_train, y_train, batch_size=1, epochs=1)
    
    # Demonstrate prediction (for illustration; expand as needed)
    test_data = scaled_data[training_data_len - 60:, :]
    x_test = []
    for i in range(60, len(test_data)):
        x_test.append(test_data[i-60:i, 0])
    x_test = np.array(x_test)
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))
    
    predictions = model.predict(x_test)
    predictions = scaler.inverse_transform(predictions)  # Inverse transform to get actual value
    
    # Plotting the data
    train = data[:training_data_len]
    valid = data[training_data_len:]
    valid['Predictions'] = predictions
    plt.figure(figsize=(16,8))
    plt.title('Model')
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD ($)', fontsize=18)
    plt.plot(train['Close'])
    plt.plot(valid[['Close', 'Predictions']])
    plt.legend(['Train', 'Val', 'Predictions'], loc='lower right')
    plt.show()

In [ ]:
if __name__ == "__main__":
    file_path = "stock_prices.csv"
    run_stock_price_forecasting(file_path)